In [1]:
import pandas as pd

In [2]:
news_dataset = pd.read_csv('train.csv')

In [3]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
news_dataset = news_dataset.fillna('')

In [6]:
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [7]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [9]:
port_stem = PorterStemmer()

In [10]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [11]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [12]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [13]:
X = news_dataset['content'].values
y = news_dataset['label'].values

In [14]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [15]:
print(y)

[1 0 1 ... 0 1 1]


In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [17]:
voc_size=10000
corpus=X
print(corpus)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[5477, 9531, 3253, 1242, 1067, 2677, 334, 6852, 2405, 4895, 6707, 335],
 [6680, 1150, 8780, 8780, 3038, 1384, 1681, 9237, 9907, 5996],
 [4416, 6184, 2232, 156, 3861, 7059],
 [8832, 6243, 3402, 8970, 5078, 1528, 8177, 7613],
 [3669, 980, 581, 9237, 2941, 9991, 6207, 7750, 9237, 1453, 8731, 5025],
 [6680,
  9673,
  3799,
  9839,
  1688,
  521,
  809,
  5296,
  3561,
  4312,
  4933,
  9113,
  8042,
  8692,
  5244,
  6909,
  5996],
 [8676, 8676, 4804, 3509, 6674, 9840, 5562, 3120, 6237, 7809, 6863, 1167],
 [2925,
  1150,
  5095,
  1538,
  5240,
  995,
  5801,
  7219,
  1668,
  2255,
  3428,
  5314,
  981,
  6410],
 [199, 2693, 8416, 5263, 5296, 1240, 1161, 1193, 734, 3803, 5314, 981, 6410],
 [801,
  1894,
  262,
  885,
  2797,
  3531,
  74,
  3417,
  5236,
  3050,
  5296,
  4399,
  5314,
  981,
  6410],
 [9906, 8268, 6063, 8564, 4275, 5022, 2971, 995, 676, 1303, 5296, 7911],
 [5872, 3895, 1533, 2936, 3881, 5749, 2470, 228, 8026, 1705],
 [5323, 672, 4891, 1080, 6477, 4253, 80, 6169, 6663, 

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4895 6707  335]
 [   0    0    0 ... 9237 9907 5996]
 [   0    0    0 ...  156 3861 7059]
 ...
 [   0    0    0 ... 5314  981 6410]
 [   0    0    0 ... 6093 8248 5721]
 [   0    0    0 ... 3880 5768 4738]]


In [20]:
len(embedded_docs),y.shape

(20800, (20800,))

In [21]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [22]:
X_final.shape,y_final.shape

((20800, 20), (20800,))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 24s 60ms/step - loss: 0.2421 - accuracy: 0.8855 - val_loss: 0.0494 - val_accuracy: 0.9843
Epoch 2/10
218/218 [==============================] - 11s 52ms/step - loss: 0.0253 - accuracy: 0.9920 - val_loss: 0.0442 - val_accuracy: 0.9851
Epoch 3/10
218/218 [==============================] - 11s 51ms/step - loss: 0.0103 - accuracy: 0.9977 - val_loss: 0.0329 - val_accuracy: 0.9910
Epoch 4/10
218/218 [==============================] - 11s 51ms/step - loss: 0.0080 - accuracy: 0.9980 - val_loss: 0.0283 - val_accuracy: 0.9923
Epoch 5/10
218/218 [==============================] - 11s 50ms/step - loss: 0.0029 - accuracy: 0.9995 - val_loss: 0.0316 - val_accuracy: 0.9913
Epoch 6/10
218/218 [==============================] - 11s 51ms/step - loss: 0.0017 - accuracy: 0.9994 - val_loss: 0.0301 - val_accuracy: 0.9916
Epoch 7/10
218/218 [==============================] - 11s 50ms/step - loss: 8.7144e-04 - accuracy: 0.9998 - val_loss: 0.0342 - val_accur

In [28]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)


215/215 [==============================] - 14s 14ms/step


In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(y_test,y_pred)


array([[3431,   18],
       [  32, 3383]], dtype=int64)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9927156177156177

In [32]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            400000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=2)

In [33]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 53s 131ms/step - loss: 0.1677 - accuracy: 0.9301 - val_loss: 0.0477 - val_accuracy: 0.9892
Epoch 2/10
218/218 [==============================] - 14s 64ms/step - loss: 0.0185 - accuracy: 0.9947 - val_loss: 0.0316 - val_accuracy: 0.9913
Epoch 3/10
218/218 [==============================] - 14s 67ms/step - loss: 0.0075 - accuracy: 0.9983 - val_loss: 0.0437 - val_accuracy: 0.9881
Epoch 4/10
218/218 [==============================] - 18s 83ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.0515 - val_accuracy: 0.9881
Epoch 5/10
218/218 [==============================] - 17s 80ms/step - loss: 4.3919e-04 - accuracy: 0.9999 - val_loss: 0.0582 - val_accuracy: 0.9879
Epoch 6/10
218/218 [==============================] - 15s 69ms/step - loss: 0.0026 - accuracy: 0.9992 - val_loss: 0.0742 - val_accuracy: 0.9822
Epoch 7/10
218/218 [==============================] - 15s 67ms/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.1439 - val_accu

In [34]:
y_pred1 = model1.predict(X_test)
y_pred1 = np.round(y_pred1).astype(int)

215/215 [==============================] - 5s 13ms/step


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[3427,   22],
       [  38, 3377]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9912587412587412